Importing Model

In [1]:
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

ckpt = torch.load("/content/Latest_model.pth", map_location=device)

Defining Class Architecture


In [2]:
import torch.nn as nn
import torch.nn.functional as F

class SEBlock(nn.Module):
    def __init__(self, channels, reduction=8):
        super().__init__()
        self.fc1 = nn.Linear(channels, channels//reduction)
        self.fc2 = nn.Linear(channels//reduction, channels)

    def forward(self, x):
        b,c,h,w = x.shape
        y = x.mean(dim=(2,3))
        y = F.relu(self.fc1(y))
        y = torch.sigmoid(self.fc2(y))
        y = y.view(b,c,1,1)
        return x * y


class NoiseRobustCNN(nn.Module):
    def __init__(self, num_classes=7):
        super().__init__()

        self.block1 = nn.Sequential(
            nn.Conv2d(3,32,3,padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Conv2d(32,32,3,padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )

        self.block2 = nn.Sequential(
            nn.Conv2d(32,64,3,padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Conv2d(64,64,3,padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )

        self.block3 = nn.Sequential(
            nn.Conv2d(64,128,3,padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU()
        )

        self.se = SEBlock(128)

        self.classifier = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Flatten(),
            nn.Dropout(0.5),
            nn.Linear(128, num_classes)
        )

    def forward(self,x):
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = self.se(x)
        x = self.classifier(x)
        return x


Loading the Model

In [3]:
model = NoiseRobustCNN(num_classes=ckpt["num_classes"]).to(device)
model.load_state_dict(ckpt["model_state"])
model.eval()


NoiseRobustCNN(
  (block1): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (block2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (block3): Sequential(
    (0): Conv2d(6

Loading the Dataset

In [5]:
import numpy as np

data = np.load("/content/extended_validation.npz")

x_test = data["x_val"]   # (N,28,28,3)
y_test = data["y_val"]

print(x_test.shape, y_test.shape)


(4012, 28, 28, 3) (4012, 1)


Preprocessing

In [6]:
import numpy as np
x_test_np = data["x_val"]
y_test_np = data["y_val"]

x_test = torch.tensor(x_test_np/ckpt["scale"], dtype=torch.float32)
x_test = x_test.permute(0,3,1,2)
y_test = torch.tensor(y_test_np.squeeze(), dtype=torch.long)

print(x_test.shape, y_test.shape)

torch.Size([4012, 3, 28, 28]) torch.Size([4012])


Creating a dataloader

In [7]:
from torch.utils.data import DataLoader, TensorDataset

test_loader = DataLoader(
    TensorDataset(x_test, y_test),
    batch_size=64,
    shuffle=False
)


Evaluating the model

In [8]:
from sklearn.metrics import accuracy_score, classification_report

all_preds, all_labels = [], []

with torch.no_grad():
    for imgs, labels in test_loader:
        imgs = imgs.to(device)
        outputs = model(imgs)
        preds = outputs.argmax(1).cpu().numpy()

        all_preds.extend(preds)
        all_labels.extend(labels.numpy())

print("Accuracy:", accuracy_score(all_labels, all_preds))
print(classification_report(all_labels, all_preds))


Accuracy: 0.6415752741774676
              precision    recall  f1-score   support

           0       0.13      0.12      0.13       132
           1       0.28      0.45      0.35       208
           2       0.47      0.27      0.34       440
           3       0.02      0.02      0.02        48
           4       0.41      0.42      0.42       444
           5       0.88      0.79      0.83      2684
           6       0.12      0.80      0.21        56

    accuracy                           0.64      4012
   macro avg       0.33      0.41      0.33      4012
weighted avg       0.71      0.64      0.66      4012



Finding out classwise accuracy

In [9]:
import numpy as np

def classwise_accuracy(model, dataloader, num_classes=7):
    model.eval()

    correct = np.zeros(num_classes)
    total   = np.zeros(num_classes)

    with torch.no_grad():
        for imgs, labels in dataloader:
            imgs, labels = imgs.to(device), labels.to(device)

            outputs = model(imgs)
            preds = outputs.argmax(1)

            for i in range(len(labels)):
                label = labels[i].item()
                pred  = preds[i].item()

                if pred == label:
                    correct[label] += 1
                total[label] += 1

    for c in range(num_classes):
        acc = correct[c] / total[c] if total[c] > 0 else 0
        print(f"Class {c} Accuracy: {acc:.4f}")


In [ ]:
classwise_accuracy(model, test_loader)


Class 0 Accuracy: 0.1742
Class 1 Accuracy: 0.4904
Class 2 Accuracy: 0.3136
Class 3 Accuracy: 0.0833
Class 4 Accuracy: 0.5428
Class 5 Accuracy: 0.7090
Class 6 Accuracy: 0.4286
